In [1]:
import pandas as pd
import numpy as np

In [2]:
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

In [3]:
import requests
from requests_oauthlib import OAuth1

In [4]:
import cryptography
from cryptography.fernet import Fernet

In [5]:
from scipy.sparse import csr_matrix

In [6]:
from sklearn.externals import joblib

In [7]:
from sklearn.preprocessing import LabelEncoder

In [8]:
import datetime as dt

In [9]:
import random

In [10]:
import pickle

In [11]:
from Candidates import candidates_list

In [12]:
with open ('../config/config_key.key', 'rb') as ck:
    fernet_key = ck.read()

with open('../config/config_encrypt_1.key', 'rb') as c1:
    cke_e = c1.read()
with open('../config/config_encrypt_2.key', 'rb') as c2:
    cse_e = c2.read()
with open('../config/config_encrypt_3.key', 'rb') as c3:
    ate_e = c3.read()
with open('../config/config_encrypt_4.key', 'rb') as c4:
    atse_e = c4.read()

In [13]:
fernet = Fernet(fernet_key)

In [14]:
cke_d = fernet.decrypt(cke_e)
cse_d = fernet.decrypt(cse_e)
ate_d = fernet.decrypt(ate_e)
atse_d = fernet.decrypt(atse_e)

In [15]:
ck = cke_d.decode()
cs = cse_d.decode()
at = ate_d.decode()
ats = atse_d.decode()

In [16]:
# Create authorization object
auth = OAuth1(ck, cs, at, ats)

In [17]:
callback_url = "https://tweetocracy.herokuapp.com/"
payload = {
    'oauth_callback':callback_url
}

In [18]:
# Execute a POST/Auth request to twittier api to intitiate access
r = requests.post('https://api.twitter.com/oauth/request_token', auth = auth, data = payload)

In [19]:
print(f'Post Request Token URL:{r.url}')
print(f'Post Request Status:{r.status_code}')
print(f'Post Request Text: {r.text}')

Post Request Token URL:https://api.twitter.com/oauth/request_token
Post Request Status:200
Post Request Text: oauth_token=tSu_RQAAAAAA_QraAAABbNmM9oU&oauth_token_secret=1v328VMwNf915EnMfRkJC80CQh9Y6vLR&oauth_callback_confirmed=true


In [20]:
# Collect response information
response_output = r.text
# Relevant paramters are received as a string, separated by an '&' character
response_parameters = response_output.split("&")

In [21]:
# Store relevant response paramters in variables
oauth_token = response_parameters[0][12:]
print(f'OAuth_token:{oauth_token}')
oauth_token_secret=response_parameters[1][19:]
print(f'Oauth Token Secret:{oauth_token_secret}')
oauth_callback_confirmed = bool(response_parameters[2][25:])
print(f'Callback Confirmed:{oauth_callback_confirmed}')

OAuth_token:tSu_RQAAAAAA_QraAAABbNmM9oU
Oauth Token Secret:1v328VMwNf915EnMfRkJC80CQh9Y6vLR
Callback Confirmed:True


In [22]:
extended_payload = {
    'tweet_mode': 'extended'
}

In [26]:
with open('top_candidates.pkl', 'rb') as f:
   top_candidates = pickle.load(f)

In [27]:
top_candidates

['Donald J. Trump',
 'Bernie Sanders',
 'Kamala Harris',
 'Elizabeth Warren',
 'Joe Biden']

In [28]:
filtered_candidates = list(filter(lambda x: x["name"] in top_candidates, candidates_list))

In [29]:
filtered_candidates

[{'name': 'Joe Biden',
  'party': 'Democrat',
  'twitter_url': 'https://twitter.com/JoeBiden',
  'twitter_screen_name': 'JoeBiden',
  'twitter_user_id': '939091',
  'announcement_date': 'April 25, 2019',
  'status': 'running'},
 {'name': 'Kamala Harris',
  'party': 'Democrat',
  'twitter_url': 'https://twitter.com/KamalaHarris',
  'twitter_screen_name': 'KamalaHarris',
  'twitter_user_id': '30354991',
  'announcement_date': 'January 21, 2019',
  'status': 'running'},
 {'name': 'Bernie Sanders',
  'party': 'Independent',
  'twitter_url': 'https://twitter.com/BernieSanders',
  'twitter_screen_name': 'BernieSanders',
  'twitter_user_id': '216776631',
  'announcement_date': 'February 19, 2019',
  'status': 'running'},
 {'name': 'Donald J. Trump',
  'party': 'Republican',
  'twitter_url': 'https://twitter.com/realDonaldTrump',
  'twitter_screen_name': 'realDonaldTrump',
  'twitter_user_id': '25073877',
  'announcement_date': 'January 20, 2017',
  'status': 'running'},
 {'name': 'Elizabeth W

In [30]:
random_candidate = random.choice(filtered_candidates)
candidate_id = random_candidate["twitter_user_id"]

In [31]:
user_get = requests.get(f'https://api.twitter.com/1.1/statuses/user_timeline.json?id={candidate_id}&count=100', params = extended_payload, auth = auth)


In [32]:
user_json = user_get.json()

In [33]:
def filter_aged(list_element):
    date_string = list_element["created_at"]
    datetime_object = dt.datetime.strptime(date_string, "%a %b %d %H:%M:%S %z %Y")
    date_object = datetime_object.date()
    
    today_datetime = dt.datetime.utcnow()
    today_date = today_datetime.date()
    two_days_prior = today_date - dt.timedelta(days = 2)

    training_data = dt.date(2019, 8, 14)
    
    return ((date_object <= two_days_prior) and (date_object > training_data)) 

#     return date_object > training_data

In [34]:
user_filtered = list(filter(lambda x: filter_aged(x), user_json))

In [35]:
len(user_filtered)

71

In [36]:
tweet_selection = random.choice(user_filtered)

In [37]:
tweet_selection

{'created_at': 'Sun Aug 25 15:47:31 +0000 2019',
 'id': 1165651928400838656,
 'id_str': '1165651928400838656',
 'full_text': 'Happy Birthday Sean! https://t.co/iQgY6RTWST',
 'truncated': False,
 'display_text_range': [0, 20],
 'entities': {'hashtags': [],
  'symbols': [],
  'user_mentions': [],
  'urls': [{'url': 'https://t.co/iQgY6RTWST',
    'expanded_url': 'https://twitter.com/chc_1927/status/1165526646448951297',
    'display_url': 'twitter.com/chc_1927/statu…',
    'indices': [21, 44]}]},
 'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>',
 'in_reply_to_status_id': None,
 'in_reply_to_status_id_str': None,
 'in_reply_to_user_id': None,
 'in_reply_to_user_id_str': None,
 'in_reply_to_screen_name': None,
 'user': {'id': 25073877,
  'id_str': '25073877',
  'name': 'Donald J. Trump',
  'screen_name': 'realDonaldTrump',
  'location': 'Washington, DC',
  'description': '45th President of the United States of America🇺🇸',
  'url': 'https://t.co

In [38]:
tweet_dict = {
    'full_text': tweet_selection["full_text"],
    'retweet_count': tweet_selection["retweet_count"],
    'favorite_count': tweet_selection['favorite_count'],
    'created_at': tweet_selection['created_at']
}

In [39]:
def convert_day(date_string):
    datetime_object = dt.datetime.strptime(date_string, "%a %b %d %H:%M:%S %z %Y")
    day = dt.datetime.strftime(datetime_object, "%A")
    return day
def convert_hour(date_string):
    datetime_object = dt.datetime.strptime(date_string, "%a %b %d %H:%M:%S %z %Y")
    hour = dt.datetime.strftime(datetime_object, "%H")
    return hour
def convert_month(date_string):
    datetime_object = dt.datetime.strptime(date_string, "%a %b %d %H:%M:%S %z %Y")
    month = dt.datetime.strftime(datetime_object, "%B")
    return month

In [40]:
tweet_dict['day'] = convert_day(tweet_dict['created_at'])
tweet_dict['hour'] = convert_hour(tweet_dict['created_at'])
tweet_dict['month']  = convert_month(tweet_dict['created_at'])

In [41]:
tweet_dict

{'full_text': 'Happy Birthday Sean! https://t.co/iQgY6RTWST',
 'retweet_count': 12918,
 'favorite_count': 74678,
 'created_at': 'Sun Aug 25 15:47:31 +0000 2019',
 'day': 'Sunday',
 'hour': '15',
 'month': 'August'}

In [42]:
tweet_list = [tweet_dict]

In [43]:
tweet_df = pd.DataFrame(tweet_list)
tweet_df.head()

,created_at,day,favorite_count,full_text,hour,month,retweet_count
0,Sun Aug 25 15:47:31 +0000 2019,Sunday,74678,Happy Birthday Sean! https://t.co/iQgY6RTWST,15,August,12918


In [44]:
import nltk
import re
import string
pd.set_option('display.max_colwidth', 100) # To extend column width

stopwords = nltk.corpus.stopwords.words('english')
wn = nltk.WordNetLemmatizer()

In [45]:
def clean_text(text):
    text = text.replace('&amp;', '&')
    text = text.replace('\n', ' ')
    text = "".join([word.lower() for word in text if word not in string.punctuation])
    tokens = re.split('\W+', text)
    text = [wn.lemmatize(word) for word in tokens if word not in stopwords]
    return text

In [46]:
from sklearn.feature_extraction.text import CountVectorizer

# NGramVectorizer
ngram_vect = CountVectorizer(ngram_range=(2,2), analyzer=clean_text)
ngram_vect.fit_transform(tweet_df['full_text'])

<1x4 sparse matrix of type '<class 'numpy.int64'>'
	with 4 stored elements in Compressed Sparse Row format>

In [47]:
ngram_vect.get_feature_names()

['birthday', 'happy', 'httpstcoiqgy6rtwst', 'sean']

In [48]:
with open('rf_columns.pkl', 'rb') as f:
   columns_list = pickle.load(f)

In [49]:
null_list = []
for i in range(0, len(columns_list)):
    null_list.append(0)

In [50]:
X_features = dict(zip(columns_list, null_list))

In [51]:
X_features['retweet_count'] = tweet_dict['retweet_count']
X_features['favorite_count'] = tweet_dict['favorite_count']

select_month = tweet_dict['month']
select_day = tweet_dict['day']
select_hour = tweet_dict['hour']
X_features[f'month_{select_month}'] = 1
X_features[f'day_{select_day}'] = 1
X_features[f'hour_{select_hour}'] = 1

In [52]:
for word in ngram_vect.get_feature_names():
    if word in X_features.keys():
        X_features[word] += 1

In [53]:
len(list(X_features.values()))

18591

In [55]:
X_features = np.array(list(X_features.values())).reshape((1, 18591))

In [56]:
X_features.shape

(1, 18591)

In [57]:
X_features

array([[12918, 74678,     0, ...,     0,     0,     0]])

In [58]:
X_sparse = csr_matrix(X_features)

In [59]:
X_sparse

<1x18591 sparse matrix of type '<class 'numpy.int32'>'
	with 8 stored elements in Compressed Sparse Row format>

In [60]:
X_sparse.toarray()

array([[12918, 74678,     0, ...,     0,     0,     0]], dtype=int32)

In [61]:
scaler_filename = "rf_scaler.save"
scaler = joblib.load(scaler_filename) 

In [62]:
X_scaled = scaler.transform(X_sparse)

In [63]:
encoder = LabelEncoder()
encoder.classes_ = np.load('rf_classes.npy')

In [64]:
with open('rf_model.sav', 'rb') as f:
   model = pickle.load(f)

In [65]:
prediction = model.predict(X_scaled)
prediction_label = encoder.inverse_transform(prediction)
prediction_label

array(['Donald J. Trump'], dtype=object)

In [66]:
prediction

array([1])

In [67]:
encoder.classes_

array(['Bernie Sanders', 'Donald J. Trump', 'Elizabeth Warren',
       'Joe Biden', 'Kamala Harris'], dtype=object)

In [68]:
prediction_prob = model.predict_proba(X_scaled)
prediction_prob = [float(i) for i in prediction_prob[0]]
classes_prob = list(zip(prediction_prob, encoder.classes_))
sorted_class = sorted(classes_prob, key = lambda x: x[0], reverse = True)
sorted_top = sorted_class[0:2]

In [69]:
sorted_top

[(0.8676470588235294, 'Donald J. Trump'), (0.0661764705882353, 'Joe Biden')]